In [1]:
# Import Libaries and Packages
import numpy as np
import pandas as pd
import re

## Clean Articles Final

This noteboks performs the following procedures:
    
    1. Reads in extracted and cleanded data from prior processes as a csv. 
    2. Performs addtional preprocessing.
    3. Discards data that does not contain selected keywords.

In [2]:
articles_raw = pd.read_csv("use_rEx.csv") #Do not run more than once - hogs memory and takes some time
articles_raw.head()

,author,contents,description,publisher,source_url,timeStamp,title
14599,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
14591,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
14592,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
14593,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
14594,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [3]:
articles_raw.tail()

,author,contents,description,publisher,source_url,timeStamp,title
28959,Avivah Litan,\tA member of the Gartner Blog Network\tby Avi...,Blockchain sessions were in full force at the ...,Gartner.com,https://blogs.gartner.com/avivah-litan/2018/04...,2018-04-23 23:22:25+00:00,"Blockchain, Crypto, and Women; Musings from RSAC"
28960,Cyberparse.co.uk,Enlarge / Simplified figurative process of a C...,Enlarge / Simplified figurative process of a C...,Cyberparse.co.uk,http://cyberparse.co.uk/2018/04/23/new-hacks-s...,2018-04-23 23:32:00+00:00,New hacks siphon private cryptocurrency keys f...
28961,Ali Breland and Harper Neidig,THE EU'S NEW TARGET: They're back! European re...,THE EU'S NEW TARGET: They're back! European re...,The Hill,http://thehill.com/policy/technology/overnight...,2018-04-23 23:49:29+00:00,Overnight Tech: EU investigates Apple's Shazam...
28962,Tyler Durden,Surprise! Even kids can be taught about finan...,"Authored by Caitlin Johnstone via Medium.com, ...",Zerohedge.com,https://www.zerohedge.com/news/2018-04-23/msm-...,2018-04-23 23:55:00+00:00,MSM Is Frantically Attacking Dissenting Syria ...
28963,pjbyrne,"Disclaimer: English lawyer, not practising thi...","Disclaimer: English lawyer, not practising thi...",Prestonbyrne.com,https://prestonbyrne.com/2018/04/23/on-ethereu...,2018-04-23 23:59:35+00:00,Whether Ethereum is a security


In [4]:
articles_raw.describe()

,author,contents,description,publisher,source_url,timeStamp,title
count,28601,28774,28770,28774,28774,28774,28764
unique,7079,26417,27539,1740,27733,26813,25302
top,Scott Scanlon,If you are the site owner (or you manage this ...,View on Dribbble,Youbrandinc.com,https://www.bloomberg.com/news/articles/2018-0...,2018-03-26 12:00:00+00:00,6 things Australian traders will be talking ab...
freq,1743,57,5,1730,3,10,43


__Preprocess and Clean Data__

In [5]:
#Reset index
articles_raw = articles_raw.reset_index()
articles_raw = articles_raw.drop('index', axis = 1)
articles_raw.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


#### Methodology: 
We have a list of keywords (defined below) that we deem relevant to Bitcoin trading and price. If an article does not mention one of these keywords in the first 300 characters (heuristic/empirical analysis of where the lede of an article ends) we deem it irrelevant.

In [6]:
#Sample of first 300 characters of the first 30 articles. 
#As you can see, relevant articles will mention Bitcoin or a keyword in this selection.
for article in articles_raw['contents'][:30]:
    print(article[:300])
    print()

A complete payments platform engineered for growth.	Build and scale your recurring business model.	Everything platforms need to get sellers paid.	Your business data at your fingertips.	The best way to start an internet business.	Fight fraud with machine learning.	Share this post on Twitter	
Tom Karl

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times local):	11:20 p.m.
  	President Donald Trump plans to meet with world leaders, dine with business executives and declare America "open for business" during his trip to the Davos summit.	White House official

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times local):	11:20 p.m.
  	President Donald Trump plans to meet with world leaders, dine with business executives and declare America "open for business" during his trip to the Davos summit.	White House official

DAVOS, Switzerland  — The latest on the World Economic Forum in Davos, Switzerland (all times 

__Filtering Keyword List__

In [7]:
##########Find a corpora
articles_final = articles_raw

In [8]:

# import packages

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

In [9]:
# 1. 
from nltk.corpus import stopwords


def article_cleaner(article):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    article = bs.BeautifulSoup(article).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', article)
    
    #3. Remove punctuation
    article = re.sub("[^a-zA-Z]", " ",article)
    
    #4. Tokenize into words (all lower case)
    article = article.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    article = [w for w in article if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    article = ' '.join(article+emoticons)
    # add emoticons to the end

    return(article)

In [10]:
articles_final[articles_final['description'].isnull()]

,author,contents,description,publisher,source_url,timeStamp,title
6944,alexmark,\nSpreading the truth. Empowering the people.\...,NaN,Investmentwatchblog.com,http://investmentwatchblog.com/before-it-colla...,2018-02-06 20:14:42+00:00,"Before it collapsed, BitConnect hucksters thou..."
13269,alexmark,\nSpreading the truth. Empowering the people.\...,NaN,Investmentwatchblog.com,http://investmentwatchblog.com/loaded-for-gucc...,2018-02-26 14:42:29+00:00,“Loaded For Guccifer” Aims For A Romanian Gucc...
20991,90Min,AC Milan owner Yonghong Li has failed in an at...,NaN,Si.com,https://www.si.com/soccer/2018/03/26/italian-r...,2018-03-26 16:50:50+00:00,Italian Report Reveals AC Milan Owners Failed ...
26610,"David Rodriguez, Quantitative Strategist, Davi...",Get the best parts of DailyFX.com in the new D...,NaN,Dailyfx.com,https://www.dailyfx.com/forex/technical/ssi/sp...,2018-04-16 16:46:00+00:00,Majority of Traders Remain Heavily Long Bitcoi...


In [11]:
%%time

num_articles = len(articles_final['contents'])

article_clean_original = []

for i in range(0,num_articles):
    if( (i+1)%1000 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    article_clean_original.append(article_cleaner(articles_final['contents'][i]))

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Done with 1000 reviews
Done with 2000 reviews
Done with 3000 reviews
Done with 4000 reviews
Done with 5000 reviews
Done with 6000 reviews
Done with 7000 reviews
Done with 8000 reviews
Done with 9000 reviews
Done with 10000 reviews
Done with 11000 reviews
Done with 12000 reviews
Done with 13000 reviews
Done with 14000 reviews
Done with 15000 reviews
Done with 16000 reviews
Done with 17000 reviews
Done with 18000 reviews
Done with 19000 reviews
Done with 20000 reviews
Done with 21000 reviews
Done with 22000 reviews
Done with 23000 reviews
Done with 24000 reviews
Done with 25000 reviews
Done with 26000 reviews
Done with 27000 reviews
Done with 28000 reviews
CPU times: user 40.1 s, sys: 8.26 s, total: 48.4 s
Wall time: 49.6 s


In [12]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [13]:
%%time
# Lemmatizer

article_clean_wnl = []

wnl = WordNetLemmatizer()

for i in range(0,num_articles):
    if( (i+1)%1000 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    
    wnl_stems = []
    token_tag = pos_tag(article_clean_original[i].split())
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)

    article_clean_wnl.append(' '.join(wnl_stems))

Done with 1000 reviews
Done with 2000 reviews
Done with 3000 reviews
Done with 4000 reviews
Done with 5000 reviews
Done with 6000 reviews
Done with 7000 reviews
Done with 8000 reviews
Done with 9000 reviews
Done with 10000 reviews
Done with 11000 reviews
Done with 12000 reviews
Done with 13000 reviews
Done with 14000 reviews
Done with 15000 reviews
Done with 16000 reviews
Done with 17000 reviews
Done with 18000 reviews
Done with 19000 reviews
Done with 20000 reviews
Done with 21000 reviews
Done with 22000 reviews
Done with 23000 reviews
Done with 24000 reviews
Done with 25000 reviews
Done with 26000 reviews
Done with 27000 reviews
Done with 28000 reviews
CPU times: user 13min 55s, sys: 4.97 s, total: 14min
Wall time: 14min 15s


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [15]:
test=pd.DataFrame(data=article_clean_wnl)#there are three columns of data, named 1, 2 and 3
print(test)
test.to_csv('article_clean_wnl.csv')

                                                       0
0      complete payment platform engineer growth buil...
1      davos switzerland late world economic forum da...
2      davos switzerland late world economic forum da...
3      davos switzerland late world economic forum da...
4      davos switzerland late world economic forum da...
5      davos switzerland late world economic forum da...
6      davos switzerland late world economic forum da...
7      davos switzerland late world economic forum da...
8      davos switzerland late world economic forum da...
9      davos switzerland late world economic forum da...
10     scramble serve massively expand userbase crypt...
11     many cryptocurrencies much money make lose mar...
12     press association publish edt january update e...
13     initial coin offering icos regulatory wild wes...
14     hannah murphy london bitflyer tokyo base opera...
15     topix user yet forget password news p p contin...
16     var postloadfunctions va

In [16]:
article_clean_wnl = pd.read_csv('article_clean_wnl.csv')

In [17]:
article_clean_wnl['0'][0:100]

0     complete payment platform engineer growth buil...
1     davos switzerland late world economic forum da...
2     davos switzerland late world economic forum da...
3     davos switzerland late world economic forum da...
4     davos switzerland late world economic forum da...
5     davos switzerland late world economic forum da...
6     davos switzerland late world economic forum da...
7     davos switzerland late world economic forum da...
8     davos switzerland late world economic forum da...
9     davos switzerland late world economic forum da...
10    scramble serve massively expand userbase crypt...
11    many cryptocurrencies much money make lose mar...
12    press association publish edt january update e...
13    initial coin offering icos regulatory wild wes...
14    hannah murphy london bitflyer tokyo base opera...
15    topix user yet forget password news p p contin...
16    var postloadfunctions var foresee enable var d...
17    real time price vires numeris bitcoin ethe

In [18]:
article_clean_wnl['0'].dropna(axis = 0,inplace = True)

In [19]:
bag_of_words = vectorizer.fit(article_clean_wnl['0'][:10000].tolist())
bag_of_words = vectorizer.transform(article_clean_wnl['0'][:10000].tolist())
words_cor = pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names())
words_cor.sum().sort_values(ascending = False)
asc=words_cor.sum().sort_values(ascending = False).index

In [20]:
asc

Index(['bitcoin', 'say', 'market', 'cryptocurrency', 'use', 'blockchain',
       'function', 'one', 'year', 'new',
       ...
       'dmi', 'dmini', 'prionsias', 'prinzen', 'printsec', 'dmjukt',
       'printouts', 'dml', 'printout', 'invoiceless'],
      dtype='object', length=72780)

In [21]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [22]:
filtered = [w for w in asc if not w in stop_words]
freqs = words_cor[filtered].sum().to_frame("freq")
freqs = freqs[:100]

freqs

,freq
bitcoin,45680
say,31286
market,23652
cryptocurrency,21723
use,21045
blockchain,19859
function,19065
one,18170
year,18105
new,17314


In [23]:
freqs.to_csv('corpora10_27.csv')

In [24]:
articles_final.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [27]:
#articles_final = articles_final.drop(indices_to_drop)

In [28]:
articles_final = articles_final.reset_index()
articles_final = articles_final.drop('index', axis = 1)
articles_final.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trudeau announces pacific trade de...
2,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: Trump to tell Davos the US 'open f...
3,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: VR film in Davos highlights cyber ...
4,"Associated Press, By Associated Press","DAVOS, Switzerland — The latest on the World ...","DAVOS, Switzerland — The Latest on the World E...",Bostonherald.com,http://www.bostonherald.com/business/business_...,2018-01-23 00:00:00+00:00,The Latest: IMF's Lagarde faults Modi speech i...


In [29]:
articles_final.to_csv("final_articles.csv")

___